# Video Object Tracking Example

This notebook demonstrates single-object tracking in a video using OpenCV's tracking API. We will select an object in the first frame and then use the **CSRT tracker** to follow it through subsequent frames. To run this, you'll need a video file or webcam feed. (See https://www.geeksforgeeks.org/getting-started-with-object-tracking-using-opencv/ for more details on trackers.)


In [1]:
# Install OpenCV if not already installed
!pip install opencv-contrib-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 4.7 MB/s  0:00:15m0:00:0100:01


In [2]:
import cv2
import numpy as np

## Initialize Video and Tracker

We open the video source (a file or camera) and read the first frame. Then we select a Region of Interest (ROI) for the object we want to track. Use the mouse to draw a box around the object in the first frame.

In [3]:
# Open video file or webcam
video_path = 'path/to/your/video.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)  # try to fall back to webcam
    print("Using webcam for tracking...")

# Read first frame
ret, frame = cap.read()
if not ret:
    print("Cannot read video source")
    cap.release()
else:
    # Let user select ROI on the first frame
    roi = cv2.selectROI("Select Object", frame, showCrosshair=True)
    cv2.destroyWindow("Select Object")
    x, y, w, h = roi
    if w == 0 or h == 0:
        print("ROI selection cancelled. Exiting.")
        cap.release()
    else:
        # Initialize CSRT tracker with the selected ROI
        tracker = cv2.legacy.TrackerCSRT_create()
        tracker.init(frame, roi)
        print("Tracker initialized on ROI:", roi)

Using webcam for tracking...
Cannot read video source


[ WARN:0@19.723] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@19.724] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


## Tracking Loop

Now we loop over the video frames, updating the tracker and drawing the bounding box. Press **ESC** to stop the tracking loop early.

In [4]:
if cap.isOpened():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Update tracker
        success, roi = tracker.update(frame)
        if success:
            x, y, w, h = map(int, roi)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, "Tracking", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(frame, "Lost", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
        cv2.imshow('Tracking', frame)
        if cv2.waitKey(30) & 0xFF == 27:
            break
    
    cap.release()
    cv2.destroyAllWindows()
else:
    print("Tracker was not initialized. Please run the above cell to select ROI.")

Tracker was not initialized. Please run the above cell to select ROI.


After running the loop, you should see the video with a bounding box following the object. If the tracker loses the object (for example, due to occlusion or fast movement), the label 'Lost' will appear. You can try different tracker algorithms (KCF, MIL, etc.) by replacing `TrackerCSRT_create()` with another OpenCV tracker.

> **Note:** In a Jupyter notebook, the `cv2.imshow` window might not display. If you're using VSCode or running the notebook locally, it should pop up a window. Press ESC in the video window to exit the loop. If needed, run this as a standalone Python script for full functionality.